In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

/Users/danfer/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459044803/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## hyperparameter

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## loss function

In [4]:
loss_fn = nn.CrossEntropyLoss()

## optimizer

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306913  [    0/60000]
loss: 2.299362  [ 6400/60000]
loss: 2.285856  [12800/60000]
loss: 2.273670  [19200/60000]
loss: 2.271387  [25600/60000]
loss: 2.259867  [32000/60000]
loss: 2.253962  [38400/60000]
loss: 2.242633  [44800/60000]
loss: 2.231475  [51200/60000]
loss: 2.208460  [57600/60000]
Test Error: 
 Accuracy: 50.6%, Avg loss: 2.210308 

Epoch 2
-------------------------------
loss: 2.231076  [    0/60000]
loss: 2.212330  [ 6400/60000]
loss: 2.172246  [12800/60000]
loss: 2.161394  [19200/60000]
loss: 2.155227  [25600/60000]
loss: 2.143381  [32000/60000]
loss: 2.145618  [38400/60000]
loss: 2.123523  [44800/60000]
loss: 2.110739  [51200/60000]
loss: 2.056992  [57600/60000]
Test Error: 
 Accuracy: 50.4%, Avg loss: 2.069330 

Epoch 3
-------------------------------
loss: 2.122101  [    0/60000]
loss: 2.078821  [ 6400/60000]
loss: 1.995242  [12800/60000]
loss: 1.980671  [19200/60000]
loss: 1.972361  [25600/60000]
loss: 1.961770  [32000/600